In [0]:
from IPython.display import display, Javascript
import json
from numpy.random import uniform, seed
from numpy import floor
from collections import namedtuple
import random 
import statistics

def _tickets_sold(p, demand_level, max_qty):
        quantity_demanded = floor(max(0, p - demand_level))
        return min(quantity_demanded, max_qty)

def simulate_revenue(days_left, tickets_left, pricing_function, rev_to_date=0, demand_level_min=100, demand_level_max=200, verbose=False):
    if (days_left == 0) or (tickets_left == 0):
        if verbose:
            if (days_left == 0):
                print("The flight took off today. ")
            if (tickets_left == 0):
                print("This flight is booked full.")
            print("Total Revenue: ${:.0f}".format(rev_to_date))
        return rev_to_date
    else:
        demand_level = uniform(demand_level_min, demand_level_max)
        p = pricing_function(days_left, tickets_left, demand_level)
        q = _tickets_sold(demand_level, p, tickets_left)
        if verbose:
            print("{:.0f} days before flight: "
                  "Started with {:.0f} seats. "
                  "Demand level: {:.0f}. "
                  "Price set to ${:.0f}. "
                  "Sold {:.0f} tickets. "
                  "Daily revenue is {:.0f}. Total revenue-to-date is {:.0f}. "
                  "{:.0f} seats remaining".format(days_left, tickets_left, demand_level, p, q, p*q, p*q+rev_to_date, tickets_left-q))
        return simulate_revenue(days_left = days_left-1,
                              tickets_left = tickets_left-q,
                              pricing_function=pricing_function,
                              rev_to_date=rev_to_date + p * q,
                              demand_level_min=demand_level_min,
                              demand_level_max=demand_level_max,
                              verbose=verbose)

def _save_score(score):
    message = {
        'jupyterEvent': 'custom.exercise_interaction',
        'data': {
            'learnTutorialId': 117,
            'interactionType': "check",
            'questionId': 'Aug31OptimizationChallenge',
            'outcomeType': 'Pass',
            'valueTowardsCompletion': score/10000,
            'failureMessage': None,
            'learnToolsVersion': "Testing"
        }
    }
    js = 'parent.postMessage(%s, "*")' % json.dumps(message)
    display(Javascript(js))

def score_me(pricing_function, sims_per_scenario=200):
    seed(0)
    Scenario = namedtuple('Scenario', 'n_days n_tickets')
    scenarios = [Scenario(n_days=100, n_tickets=100),
                 Scenario(n_days=14, n_tickets=50),
                 Scenario(n_days=2, n_tickets=20),
                Scenario(n_days=1, n_tickets=3),
                 ]
    scenario_scores = []
    for s in scenarios:
        scenario_score = sum(simulate_revenue(s.n_days, s.n_tickets, pricing_function)
                                     for _ in range(sims_per_scenario)) / sims_per_scenario
        print("Ran {:.0f} flights starting {:.0f} days before flight with {:.0f} tickets. "
              "Average revenue: ${:.0f}".format(sims_per_scenario,
                                                s.n_days,
                                                s.n_tickets,
                                                scenario_score))
        scenario_scores.append(scenario_score)
    score = sum(scenario_scores) / len(scenario_scores)
    try:
        _save_score(score)
    except:
        pass
    print("Average revenue across all flights is ${:.0f}".format(score))

In [0]:
# def pricing_function(days_left, tickets_left, demand_level):
    
#     """Sample pricing function"""
#     #under a KNOWN series of prices - what would be optimal? - lets try n scenarios and pick the median

#     n=10
#     #maxREV=[0]*n 
#     P=[200]*n
#     for sim_i in range(n):
#         # 100 (n) price scenarios
#         simD=[random.randrange(100, 200, 1) for i in range(days_left)] #randomly asigned future demand levels
#         simD[0]=int(demand_level) # at 0 I place the actual known demand value of today
#         #the best pricing with full information: calculate a margional revenue for a marginal ticket in each period and every time take the highest
#         simT=[0]*days_left #a list of tickets optimally sold each day
#         simMR=simD.copy() #marginal revenue from the ticket, if sold in a specific day
#         #for d_i in range(days_left):
#         #    simMR[d_i]=simD[d_i]-1
#         for ticket_i in range(int(tickets_left)):
#           if max(simMR)>0:
#                 #maxREV[sim_i]+=max(simMR)
#                 temp=simMR.index(max(simMR)) # on which day its best to sell this marginal ticket
#                 simMR[temp]=simMR[temp]-1-simT[temp] # update the marginal revenue for that day
#                 simT[temp]+=1

#         P[sim_i]=simD[0]-simT[0]
#     price=int(statistics.median(P))    
#     #    price = demand_level - 10
#     return price

    






In [0]:
### Simple function 1 - sell 10 tickets everyday

def pricing_function(days_left, tickets_left, demand_level):
    
    price = demand_level - 10
    return price


In [0]:
### Simple function 2 - sell equal number of tickets everyday (total tickets/ total days)

def pricing_function(days_left, tickets_left, demand_level):
    
    price = demand_level - tickets_left/days_left
    return price


In [0]:
### Simple function 3 - divide the tickets left by number of days left. And if the demand for that day is high - we charge a lower price and viceversa

def pricing_function(days_left, tickets_left, demand_level):
    
    price = demand_level - ((tickets_left/days_left)* (demand_level-100)/50)
    return price


In [0]:
### Simple function 4 

def pricing_function(days_left, tickets_left, demand_level):
    
    if days_left==1:  # selling all the remaining tickets
       price = demand_level-tickets_left 

    elif demand_level >= 185 and tickets_left >= demand_level/3:    #if the demand is high, we are selling half the tickets
        price = min(demand_level-(tickets_left/days_left), demand_level - (demand_level/3))

    elif demand_level >= 150: # if the demand is in the top 50%, we are selling equal number of tickets every day
        price = demand_level-(tickets_left/days_left)

    else: # if the demand is low, we are just selling one ticket that day
        price = demand_level-1

    return price

In [0]:
### Simple function 5  -  getting more selective. If we have more days, we'll sell tickets only if the demand level is high

def pricing_function(days_left, tickets_left, demand_level):

    if days_left > 14:  
        if demand_level >= 180:
            price = demand_level - ((demand_level - 100)/10)
            return price
        else:
            price = demand_level
            return price
    if days_left >= 2 and days_left <=14:
        if demand_level >= 150:
            price = demand_level - ((demand_level - 100)/ 10)
            return price
        else:
            price = demand_level
            return price
    if days_left == 1:
        price = demand_level - tickets_left
        return price



In [0]:

#### Simple function 6  -  using average and standard deviation of demand level.

demand_list = []
avrg_demand = demand_list

def avrg_calc(demand_level):
    avrg_demand.append(demand_level)
    return np.mean(avrg_demand)

def std_demand(demand_level):
    return np.std(avrg_demand)

def pricing_function(days_left, tickets_left, demand_level):
  average_demand = avrg_calc(demand_level)  #Average demand
  STD_demand = std_demand(demand_level)     #Standard deviation of demand

  if days_left > 14:
    if demand_level >= average_demand + (STD_demand):   #If demand is 1 standard deviation above the average
      price = demand_level - ((demand_level-100)/10)
      return price
    else:
      price = demand_level
      return price
  if days_left >= 2 and days_left <= 14:
    if demand_level >= average_demand + (0.5*STD_demand):  #If demand is 0.5 standard deviation above the average
      price = demand_level - ((demand_level-100)/10)
      return price
    else:
      price = demand_level
      return price
  if days_left == 1:   #Last day
    price = demand_level - tickets_left
    return price

https://www.kaggle.com/sausagelion/airline-price-optimization-micro-challenge

We decided to sell tickets only days where demand is above a certain threshold, thus we had to calculate what the ideal threshold is.

As a simple case, if we choose to sell tickets on 100% of days, the expected revenue can be written as follows:

revenue=(150−(tickets/days))∗tickets ## selling equal number of tickets every day


If we decide to sell tickets only on fraction of days, where demand is higher than a threshold, we get the following formula:

revenue=(200−50∗fraction−tickets/(days∗fraction))∗tickets
 
(upper bound of demand = 200, lower bound of demand = 200 - 100*fraction. Hence expected value is 200 - 50*fraction)

Then we need to maximise revenue with respect to fraction, where tickets and days are known constants. After taking a derivative and equating it to zero, we get the following:

fraction=sqrt(tickets/(50∗days))
 

In [0]:
### Simple function 7

def pricing_function(days_left, tickets_left, demand_level):

    if days_left == 1:
        price = demand_level - tickets_left
        #price = demand_level/2
    else:
        #we will sell tickets only if demand crosses a threshold
        #the optimum demand is calculated by checking where the derivative reaches maximum
        fraction = (tickets_left / (50 * days_left)) ** 0.5
        demand_treshold = int(200 - fraction*100)
        if demand_level >= demand_treshold:
            tickets_to_sell = int(tickets_left / (days_left * fraction))
            price = demand_level - tickets_to_sell
        else:
            #if the demand is not above the threshold, we are not selling any tickets
            price = demand_level
    return price


In [0]:
simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function, verbose=True)

7 days before flight: Started with 50 seats. Demand level: 159. Price set to $159. Sold 0 tickets. Daily revenue is 0. Total revenue-to-date is 0. 50 seats remaining
6 days before flight: Started with 50 seats. Demand level: 174. Price set to $154. Sold 20 tickets. Daily revenue is 3080. Total revenue-to-date is 3080. 30 seats remaining
5 days before flight: Started with 30 seats. Demand level: 138. Price set to $138. Sold 0 tickets. Daily revenue is 0. Total revenue-to-date is 3080. 30 seats remaining
4 days before flight: Started with 30 seats. Demand level: 151. Price set to $151. Sold 0 tickets. Daily revenue is 0. Total revenue-to-date is 3080. 30 seats remaining
3 days before flight: Started with 30 seats. Demand level: 189. Price set to $167. Sold 22 tickets. Daily revenue is 3670. Total revenue-to-date is 6749. 8 seats remaining
2 days before flight: Started with 8 seats. Demand level: 165. Price set to $165. Sold 0 tickets. Daily revenue is 0. Total revenue-to-date is 6749. 8 

7913.846123535096

In [0]:
score_me(pricing_function)

Ran 200 flights starting 100 days before flight with 100 tickets. Average revenue: $18390
Ran 200 flights starting 14 days before flight with 50 tickets. Average revenue: $8408
Ran 200 flights starting 2 days before flight with 20 tickets. Average revenue: $2806
Ran 200 flights starting 1 days before flight with 3 tickets. Average revenue: $447


<IPython.core.display.Javascript object>

Average revenue across all flights is $7513


Model 8: Dynamic Programming:

In [0]:
import numpy as np
import copy

def pricing_function (days_left, tickets_left, demand_level):

  t=int(tickets_left)
  d=int(days_left)
  dl=int(demand_level)

  dValues=np.linspace(start=0, stop=d, num=d+1)
  tValues=np.linspace(start=0, stop=t, num=t+1)
  dN=len(dValues)
  tN=len(tValues)

  V = np.zeros( (dN, tN) )
  U = copy.deepcopy(V)
  for d in (dValues):
    for t in (tValues):
    #for each time value the loop through the possible values of Tickets
    #cat(d,t)
      if(d==0):
        # Boundary condition
        V[int(d), int(t)]=0
      elif(t==0):
        # Boundary Condition 
        V[int(d), int(t)]=0
      else:
        # Bellman Equation
          p = np.linspace(start=(dl-t), stop=dl, num=dl+1) # checking for all values of prices less than demand level
          valueChoices=np.linspace(start=(dl-t), stop=dl, num=dl+1) # Storing the value for each price
          for index in range(len(p)):
            valueChoices[index]= p[index]*max(min(t,dl-p[index]),0) +V[int(d-1),int(t-max(min(t,dl-p[index]),0))]
          #print(valueChoices)
          #print(d,t,max(valueChoices))
          V[int(d), int(t)]=max(valueChoices)
          #print(V)
          U[int(d), int(t)]=p[np.argmax(valueChoices)]
  #print(V)

  return(U[int(dN-1),int(tN-1)])
  


In [0]:
pricing_function(10,10,100)

99.0

Creating U matrix for all combinations:

When the pricing function above is used directly for simulating the optimal prices, for each and every scenario of the dl around d*t*dl number of calculations are performed which made the entire program computationally intense. There is a lot of repetition of calculation for optimal prices, which can be avoided if a matrix with all possible values is created and the pricing function just pulls the respective record needed for the corresponding d, t, dl values, In the coming sections we created an U matrix which is used to evaluate the optimal pricing for all possible combinations and pricing_function pulls the respective record.

* - This creation of U Matrix will take around 20 min to 30 minutes to run



In [0]:
t=100
d=100
dl=200

dValues=np.linspace(start=0, stop=d, num=d+1)
tValues=np.linspace(start=0, stop=t, num=t+1)
dlvalues=np.linspace(start=100, stop=dl, num=101)
dN=len(dValues)
tN=len(tValues)
dlN=len(dlvalues)

V = np.zeros( (dN, tN,dlN) )
U = copy.deepcopy(V)
V

for dl in (dlvalues):
  for d in (dValues):
      for t in (tValues):
      #for each time value the loop through the possible values of Tickets
      #cat(d,t)
        if(d==0):
          # Boundary condition
          V[int(d), int(t),int(dl-100)]=0
        elif(t==0):
          # Boundary Condition 
          V[int(d), int(t),int(dl-100)]=0
        else:
          # Bellman Equation
            p = np.linspace(start=int(dl-t), stop=int(dl), num=int(dl+1)) # checking for all values of prices less than demand level
            valueChoices=np.linspace(start=int(dl-t), stop=int(dl), num=int(dl+1)) # Storing the value for each price
            for index in range(len(p)):
              valueChoices[index]= p[index]*max(min(t,dl-p[index]),0) +V[int(d-1),int(t-max(min(t,dl-p[index]),0)),int(dl-100)]
            #print(valueChoices)
            #print(d,t,max(valueChoices))
            V[int(d), int(t),int(dl-100)]=max(valueChoices)
            #print(V)
            U[int(d), int(t),int(dl-100)]=p[np.argmax(valueChoices)]
  #print(V)
U




array([[[  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        ...,
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ]],

       [[  0.        ,   0.        ,   0.        , ...,   0.        ,
           0.        ,   0.        ],
        [ 99.        , 100.        , 101.        , ..., 197.        ,
         198.        , 199.        ],
        [ 98.        ,  99.        , 100.        , ..., 196.        ,
         197.        , 198.        ],
        ...,


In [0]:
def pricing_function (days_left, tickets_left, demand_level):
  t=int(tickets_left)
  d=int(days_left)
  dl=int(demand_level)
  return U[d,t,dl-100]

In [0]:
score_me(pricing_function)

Ran 200 flights starting 100 days before flight with 100 tickets. Average revenue: $14728
Ran 200 flights starting 14 days before flight with 50 tickets. Average revenue: $7230
Ran 200 flights starting 2 days before flight with 20 tickets. Average revenue: $2845
Ran 200 flights starting 1 days before flight with 3 tickets. Average revenue: $438


<IPython.core.display.Javascript object>

Average revenue across all flights is $6310


In [0]:
simulate_revenue(days_left=7, tickets_left=50, pricing_function=pricing_function, verbose=True)

7 days before flight: Started with 50 seats. Demand level: 117. Price set to $111. Sold 6 tickets. Daily revenue is 666. Total revenue-to-date is 666. 44 seats remaining
6 days before flight: Started with 44 seats. Demand level: 180. Price set to $169. Sold 11 tickets. Daily revenue is 1859. Total revenue-to-date is 2525. 33 seats remaining
5 days before flight: Started with 33 seats. Demand level: 171. Price set to $165. Sold 5 tickets. Daily revenue is 825. Total revenue-to-date is 3350. 28 seats remaining
4 days before flight: Started with 28 seats. Demand level: 191. Price set to $181. Sold 9 tickets. Daily revenue is 1629. Total revenue-to-date is 4979. 19 seats remaining
3 days before flight: Started with 19 seats. Demand level: 143. Price set to $137. Sold 6 tickets. Daily revenue is 822. Total revenue-to-date is 5801. 13 seats remaining
2 days before flight: Started with 13 seats. Demand level: 187. Price set to $179. Sold 7 tickets. Daily revenue is 1253. Total revenue-to-date

8032.486163185994